In [101]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [102]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 크롤링

In [103]:
OUTPUT_FILE_NAME = 'output.txt'

In [104]:
wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"

In [105]:
import urllib.request
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [106]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [107]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = soup.select_one('h2.end_tit')
    text_r = text.get_text()
    return text_r

In [108]:
def comment(url):
    wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

# 키워드 추출

In [109]:
from krwordrank.sentence import summarize_with_sentences
def keyword_word(text):#키워드 단어 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return keywords

In [110]:
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

# 모델 로딩

In [111]:
from tensorflow.keras.models import load_model
# from konlpy.tag import Okt  
okt = Okt()

In [112]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
        max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model11.h5')
    predict = model.predict_classes(X_data)
    for i in range(len(X_data)):
        if(predict[i] == 0):
            a=0
    return a

# 플라스크

In [113]:
from flask import Flask, render_template, request
from flask import send_file, url_for
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
# import numpy as np

In [114]:
app = Flask(__name__)
url = ''

In [115]:
class ArticleForm(Form):
    articleurl = TextAreaField('',[validators.DataRequired()])

In [116]:
@app.route('/')
def index():
    form = ArticleForm(request.form)
    return render_template('article_app.html', form=form)

In [117]:
@app.route('/title', methods = ['POST'])
def article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        global url
        url = request.form['articleurl']
        title = get_title(url)
        r_title = clean_text(title)
        m_text = get_text(url)
        r_text = clean_text(m_text)
        return render_template('title.html', title=r_title, content=r_text)
    return render_template('article_app.html', form=form)

In [118]:
@app.route('/result')
def review():
    global url
    key_sentence = keyword_sentence(url)
    cr_comment = comment(url)
    result = model_load(cr_comment)
    return render_template('result.html',list=key_sentence, list2=cr_comment, list3=result)

In [119]:
@app.route('/thanks', methods = ['POST'])
def feedback():
    feedback = request.form['feedback_button']
    return render_template('thanks.html')

In [120]:
if __name__ == '__main__':
    app.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-02-20 16:25:15,372] ERROR in app: Exception on /result [GET]
Traceback (most recent call last):
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Users\kimdu\Anaconda3\lib\site-packages\